# 6. Validação Cruzada (k-Fold) da Configuração Final
Esta etapa aplica validação cruzada com $k=10$ partições na melhor configuração de rede obtida anteriormente.

Serão avaliadas as seguintes métricas em cada partição:
- Erro Relativo Médio (MRE)
- Medida-F (F1-score)
- Erro Médio Quadrático (MSE)
- Curvas de erro (treino e teste por época)

Ao final, são calculadas média e variância para comparar a robustez do modelo.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error, f1_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.initializers import GlorotUniform
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import fashion_mnist

In [ ]:
(x, y), _ = fashion_mnist.load_data()
x = x / 255.0
y = to_categorical(y, 10)

In [ ]:
def build_model():
    model = Sequential([
        Flatten(input_shape=(28, 28)),
        Dense(64, activation='relu', kernel_initializer=GlorotUniform(seed=0)),
        Dense(10, activation='softmax')
    ])
    return model

def mean_relative_error(y_true, y_pred):
    return np.mean(np.abs((y_true - y_pred) / np.maximum(y_true, 1e-8)))

## Execução da Validação Cruzada (k=10)
Para cada partição, treinamos o modelo por 20 épocas e registramos os erros.

In [ ]:
k = 10
kf = KFold(n_splits=k, shuffle=True, random_state=42)

histories = []
mre_scores = []
f1_scores = []
mse_scores = []

for i, (train_idx, test_idx) in enumerate(kf.split(x)):
    x_train, x_test = x[train_idx], x[test_idx]
    y_train, y_test = y[train_idx], y[test_idx]

    model = build_model()
    model.compile(optimizer=SGD(learning_rate=0.01),
                  loss=MeanSquaredError(),
                  metrics=['accuracy'])

    history = model.fit(x_train, y_train, epochs=20, verbose=0, validation_data=(x_test, y_test))
    histories.append(history)

    y_pred_test = model.predict(x_test)
    y_pred_classes = np.argmax(y_pred_test, axis=1)
    y_true_classes = np.argmax(y_test, axis=1)

    mre = mean_relative_error(y_true_classes, y_pred_classes)
    f1 = f1_score(y_true_classes, y_pred_classes, average='weighted')
    mse = mean_squared_error(y_true_classes, y_pred_classes)

    mre_scores.append(mre)
    f1_scores.append(f1)
    mse_scores.append(mse)

    print(f"Fold {i+1}: MRE={mre:.4f}, F1={f1:.4f}, MSE={mse:.4f}")

## Curvas de Erro por Fold
Visualizamos a evolução da perda de treino e validação por época.

In [ ]:
for i, h in enumerate(histories):
    plt.figure(figsize=(8, 4))
    plt.plot(h.history['loss'], label='Treino')
    plt.plot(h.history['val_loss'], label='Validação')
    plt.title(f'Fold {i+1} – Erro vs Época')
    plt.xlabel('Época')
    plt.ylabel('Erro')
    plt.legend()
    plt.grid(True)
    plt.show()

## Estatísticas Finais das Métricas
Abaixo, temos a média e variância dos resultados ao longo dos 10 folds.

In [ ]:
def resumo(nome, valores):
    print(f"{nome} → Média: {np.mean(valores):.4f} | Variância: {np.var(valores):.4f}")

resumo("MRE", mre_scores)
resumo("F1-score", f1_scores)
resumo("MSE", mse_scores)